In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape, Conv2D, Conv1D, MaxPooling2D, Input, Concatenate
from tensorflow.keras.models import Model
from tensorflow import keras, config
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
print("GPUs Available: ", len(config.list_physical_devices('GPU')))

In [ ]:
# Global settings
IMAGE_TARGET_SIZE = (5, 1400, 1)
BATCH_SIZE =  128

In [ ]:

%load_ext autoreload
%autoreload 2

from morse_helpers import create_sets
from morse_label_funcs import  labels_to_one_hot_positions_categorical, letter_n_to_one_hot_positions_categorical, position_regression

(image_fnames, morse_labels) = create_sets(
    [
        ["./training_data/MorseTrainSet_04/GEN04_VER_000/", 'wordsMatrices_04_000', "Words_04_000.csv"],
        ["./training_data/MorseTrainSet_04/GEN04_VER_100/", 'wordsMatrices_04_100', "Words_04_100.csv"],
        ["./training_data/MorseTrainSet_04/GEN04_VER_200/", 'wordsMatrices_04_200', "Words_04_200.csv"],
        ["./training_data/MorseTrainSet_04/GEN04_VER_210/", 'wordsMatrices_04_210', "Words_04_210.csv"],
        ["./training_data/MorseTrainSet_04/GEN04_VER_220/", 'wordsMatrices_04_220', "Words_04_220.csv"],

        ["./training_data/MorseTrainSet_11/GEN11_VER_000/", 'wordsMatrices_11_000', "Words_11_000.csv"],
        ["./training_data/MorseTrainSet_11/GEN11_VER_100/", 'wordsMatrices_11_100', "Words_11_100.csv"],
        ["./training_data/MorseTrainSet_11/GEN11_VER_200/", 'wordsMatrices_11_200', "Words_11_200.csv"],
        ["./training_data/MorseTrainSet_11/GEN11_VER_210/", 'wordsMatrices_11_210', "Words_11_210.csv"],
        ["./training_data/MorseTrainSet_11/GEN11_VER_220/", 'wordsMatrices_11_220', "Words_11_220.csv"],

        ["./training_data/MorseTrainSet_06/GEN06_VER_000/", 'wordsMatrices_06_000', "Words_06_000.csv"],
        ["./training_data/MorseTrainSet_06/GEN06_VER_100/", 'wordsMatrices_06_100', "Words_06_100.csv"],
        ["./training_data/MorseTrainSet_06/GEN06_VER_200/", 'wordsMatrices_06_200', "Words_06_200.csv"],
        ["./training_data/MorseTrainSet_06/GEN06_VER_210/", 'wordsMatrices_06_210', "Words_06_210.csv"],
        ["./training_data/MorseTrainSet_06/GEN06_VER_220/", 'wordsMatrices_06_220', "Words_06_220.csv"],

        ["./training_data/MorseTrainSet_13/GEN13_VER_000/", 'wordsMatrices_13_000', "Words_13_000.csv"],
        ["./training_data/MorseTrainSet_13/GEN13_VER_100/", 'wordsMatrices_13_100', "Words_13_100.csv"],
        ["./training_data/MorseTrainSet_13/GEN13_VER_200/", 'wordsMatrices_13_200', "Words_13_200.csv"],
        ["./training_data/MorseTrainSet_13/GEN13_VER_210/", 'wordsMatrices_13_210', "Words_13_210.csv"],
        ["./training_data/MorseTrainSet_13/GEN13_VER_220/", 'wordsMatrices_13_220', "Words_13_220.csv"],

    ], 
    IMAGE_TARGET_SIZE,
    [position_regression, letter_n_to_one_hot_positions_categorical],
    letter_n=1,
    overwrite_images=False
)

In [ ]:
# Concat label arrays
morse_labels_concat = np.array([morse_labels[0], morse_labels[1].astype("int")]).T

In [ ]:
# Improve me
def labels_to_one_hot(lbls, image_target_size = None):
    assert type(lbls[0]) == np.float64
    label_letters = lbls.astype("int")
    # clean up magic numbers
    labels_one_hot = np.zeros((label_letters.size, 26))
    labels_one_hot[np.arange(label_letters.size),label_letters] = 1
    return labels_one_hot


In [ ]:
from morse_helpers import create_all_sets

TEST_SPLIT_SIZE = 0.90
VALIDATION_SPLIT_SIZE = 0.90

train, labels, train_validation, labels_validation, train_test, labels_test = create_all_sets(
    image_fnames, morse_labels_concat, TEST_SPLIT_SIZE, VALIDATION_SPLIT_SIZE, shuffle_before_test_split=True)


In [ ]:
from morse_helpers import Image_Generator, zeropad_randomly

# TODO : create test generators with noisy images (0.1, 0.2, 0.3)

training_batch_generator = Image_Generator(train, labels, BATCH_SIZE, IMAGE_TARGET_SIZE, [zeropad_randomly], labels_to_one_hot)
validation_batch_generator = Image_Generator(train_validation, labels_validation, BATCH_SIZE, IMAGE_TARGET_SIZE, [zeropad_randomly], labels_to_one_hot)
test_batch_generator = Image_Generator(train_test, labels_test, BATCH_SIZE, IMAGE_TARGET_SIZE, [], labels_to_one_hot)


In [ ]:
# t, l = training_batch_generator.__getitem__(0)

# for i in range(10):
#     fig = plt.figure(figsize=(30,5))
#     plt.xlim(0, 800)
#     rolled = np.pad(t[i], [(0,0),(0,0), (0,0)], mode='constant')[:, 0:-1]
#     # rolled[:, int(l[i] * 1400)] = 1
#     plt.imshow(t[i])
#     plt.show()

In [ ]:
def get_addblock(x, kernelsize, filters):
    fx = layers.Conv2D(filters, kernelsize, activation='relu', padding='same')(x)
    fx = layers.BatchNormalization()(fx)
    fx = layers.Conv2D(filters, kernelsize, padding='same')(fx)
    out = layers.Add()([x,fx])
    out = layers.ReLU()(out)
    out = layers.BatchNormalization()(out)
    return out

In [ ]:
# TODO : Running this for n epochs and the validation catg_accuracy will go to 100%.
# Implement callback to save model before overfitting while validation catg_accuracy is 100% and training catg_accuracy is as high as possible.

def conv_model_catg(input_layer):
    cropped = keras.layers.Cropping2D(cropping=((0, 0), (0,1250)), data_format=None)(input_layer)
    x = keras.layers.GaussianNoise(0.15)(cropped)
    pool           = MaxPooling2D(pool_size=(2,1),padding="same")(x)

    conv1           = Conv2D(90,(3,5),padding="same",activation="relu")(pool)
    pool1           = MaxPooling2D(pool_size=(1,2),padding="same")(conv1)

    rb = get_addblock(pool1, (3,5), 90)
    pool2           = MaxPooling2D(pool_size=(1,2),padding="same")(rb)
    dropout1 = Dropout(0.2)(pool2)
    rb2 = get_addblock(dropout1, (3,7), 90)
    pool3           = MaxPooling2D(pool_size=(1,2),padding="same")(rb2)
    dropout2 = Dropout(0.1)(pool3)
    rb3 = get_addblock(dropout2, (3,3), 90)
    pool4           = MaxPooling2D(pool_size=(1,2),padding="same")(rb3)
  
    flat            = Flatten()(pool4)
    
    return flat

In [ ]:
def conv_model_pos(input_layer):
    cropped = keras.layers.Cropping2D(cropping=((0, 0), (0,1000)), data_format=None)(input_layer)
    x = keras.layers.GaussianNoise(0.05)(cropped)
    pool           = MaxPooling2D(pool_size=(2,2),padding="same")(x)

    conv1           = Conv2D(80,(3,5),padding="same",activation="relu")(pool)
    pool1           = MaxPooling2D(pool_size=(1,2),padding="same")(conv1)

    conv2           = Conv2D(80,(1,7),padding="same",activation="relu")(pool1)
    pool2           = MaxPooling2D(pool_size=(1,2),padding="same")(conv2)

    conv3           = Conv2D(80,(1,11),padding="same",activation="relu")(pool2)
    pool3           = MaxPooling2D(pool_size=(1,2),padding="same")(conv3)

    conv4           = Conv2D(80,(1,13),padding="same",activation="relu")(pool3)
    pool4           = MaxPooling2D(pool_size=(1,2),padding="same")(conv4)

    conv5           = Conv2D(80,(1,11),padding="same",activation="relu")(pool4)
    pool5           = MaxPooling2D(pool_size=(1,2),padding="same")(conv5)

    conv6           = Conv2D(80,(1,9),padding="same",activation="relu")(pool5)
    pool6           = MaxPooling2D(pool_size=(1,2),padding="same")(conv6)

    conv7           = Conv2D(80,(1,5),padding="same",activation="relu")(pool6)
    pool7           = MaxPooling2D(pool_size=(1,2),padding="same")(conv7)

    flat            = Flatten()(pool7)

    return flat

In [ ]:
from tensorflow.keras import layers

input_layer    = Input(shape=IMAGE_TARGET_SIZE)

flat_position = conv_model_pos(input_layer)
flat_letter = conv_model_catg(input_layer)

output_layer_position    = Dense(1, name="regr")(flat_position)
output_layer_letter    = Dense(26, activation="softmax", name="catg")(flat_letter)
model           = Model(inputs=input_layer, outputs=[output_layer_position, output_layer_letter])
model.compile(loss={
        "regr": "mse",
        "catg": keras.losses.CategoricalCrossentropy(),
    }, optimizer='adam', metrics={
        "regr" : "mean_absolute_error", 
        "catg" : "accuracy"
    })

print(model.summary())

init_epoch = 0

In [ ]:

NUM_EPOCHS = 15

def fit_model(epochs):
	
	global init_epoch
	history = model.fit(
					   training_batch_generator,
	                   steps_per_epoch = int(len(train) // BATCH_SIZE),
	                   epochs = epochs + init_epoch,
					   initial_epoch=init_epoch,
	                   verbose =1,
	                   validation_data = validation_batch_generator,
	                   validation_steps = int(len(train_validation) // BATCH_SIZE))
	
	
	init_epoch += epochs
	return history

history = fit_model(NUM_EPOCHS)


In [ ]:
def show_image(img):
    fig = plt.figure(figsize=(30,5))
    plt.xlim(0, 300)
    plt.imshow(img)
    plt.show()
    return fig

In [ ]:
# import sys
# np.set_printoptions(threshold=sys.maxsize)

test_gen = test_batch_generator

positions_above_px = 3

predictions = model.predict(test_gen)

def get_deviating_predictions(positions_above_px, generator, predictions):
    regression_differences = []
    categorical_differences = []
    indexer = 0

    for imgs_batch, labels_batch in generator:

        for i in range(len(imgs_batch)):

            regr_pred = predictions[0][indexer] * IMAGE_TARGET_SIZE[1]
            regr_test_label = labels_batch[0][i] * IMAGE_TARGET_SIZE[1]

            catg_pred = np.argmax(predictions[1][indexer])
            catg_test_label = np.argmax(labels_batch[1][i])

            if abs(regr_pred[0] - regr_test_label) > positions_above_px:
                regression_differences.append([regr_pred, regr_test_label, imgs_batch[i]])

            if catg_pred != catg_test_label:

                categorical_differences.append([catg_pred, catg_test_label, imgs_batch[i], predictions[1][indexer]])

            indexer += 1

    return regression_differences, categorical_differences


regression_differences, categorical_differences  = get_deviating_predictions(positions_above_px, test_gen, predictions)

print("Total predictions", len(predictions[0]))

print("Total position predictions off by more than pixels: ", positions_above_px, " : ", len(regression_differences))
print("Position predictions percentage incorrect", round( (len(regression_differences) / len(predictions[0]) * 100), 4), "%")

print("Total label predictions incorrect: ", len(categorical_differences))
print("Label predictions percentage incorrect", round( (len(categorical_differences) / len(predictions[0]) * 100), 4), "%")

In [ ]:
evaluations = model.evaluate(test_gen)
evaluations

In [ ]:
counter = 0   
for diff in categorical_differences:

    counter += 1
    if counter > 30:
        break

    pred, correct, img, one_hot = diff

    print(np.round(one_hot, 2))

    img_pred = img.copy()
    img_correct = img.copy()

    print('Prediction', pred)
    show_image(img_pred)

    print('Correct', correct)
    show_image(img_correct)

    print("----------------------------------------------------------------------------------------")

In [ ]:

counter = 0   
for diff in regression_differences:

    counter += 1
    if counter > 30:
        break

    pred, correct, img = diff

    img_pred = img.copy()
    img_correct = img.copy()

    print('Prediction', round(pred[0]))
    img_pred[:, round(int(pred))] = 1
    show_image(img_pred)

    print('Correct', round(correct))
    img_correct[:, round(int(correct))] = 1
    show_image(img_correct)

    print("----------------------------------------------------------------------------------------")


In [ ]:
import matplotlib.backends.backend_pdf as mpt

def right_roll_image_by_n(img, roll_value_n):
    return np.roll(img, roll_value_n, axis=1)

def get_label_prediction(img, model, img_width): 
    x_right_rolled_exp_dim = np.expand_dims(img, axis=0)
    single_predict = model.predict(x_right_rolled_exp_dim)
    return single_predict[0] * img_width



def roll_and_display_img(img, indexer):

    image_right_rolled = right_roll_image_by_n(img, int(-10 + (indexer / 5)))

    label_prediction = get_label_prediction(image_right_rolled, model, IMAGE_TARGET_SIZE[1])

    # Draw vertical line
    image_right_rolled[:, int(label_prediction)] = 1

    fig = show_image(image_right_rolled)
    pdf.savefig( fig )


In [ ]:
# %%capture 
# capture means suppress output

pdf = mpt.PdfPages("single_predictions_rolled_randomly.pdf")

single_image_batch, single_label_batch = test_batch_generator.__getitem__(0)

for i in range(len(single_image_batch)):
    roll_and_display_img(single_image_batch[i], i)
    
pdf.close()